**UTILITS**

This code defines a machine learning framework for a specific type of time series analysis, likely intended for anomaly detection or change point detection, using a generative adversarial network (GAN) approach. The main components and their roles are as follows:

**Environment Setup**: Imports necessary libraries and modules, loads environment variables for directory paths (e.g., model saving and predictions directories), and sets the computing device (GPU or CPU).

**Functions and Classes**:

**median_heuristic**: Calculates a set of scales based on the median of pairwise squared distances in the input data. This is often used to set the bandwidth for kernel methods, such as the Gaussian kernel in Maximum Mean Discrepancy (MMD) calculations.

**NetG and NetD**: Define the generator and discriminator networks for the GAN, using GRU (Gated Recurrent Unit) layers for sequence processing. The generator synthesizes time series data, while the discriminator attempts to distinguish between real and generated data.

**KL_CPD**: The main class implementing the GAN-based change point detection model. It includes methods for model training (fit), prediction (predict), and loss visualization (plot_losses). It utilizes MMD for comparing distributions of real and generated data as part of its training process.

**svd_wrapper**: A helper function to perform Singular Value Decomposition (SVD) using various methods (svds, randomized_svd, or np.linalg.svd). SVD is a dimensionality reduction technique that can also denoise data.

**get_reduced_data**: Reduces the dimensionality of the input dataset using SVD, which can help in processing high-dimensional time series data more efficiently.

**train_and_pred_dataset**: Orchestrates the training and prediction process for a given dataset, including handling model saving/loading based on the provided dataset name and SVD parameters.

**save_preds**: Visualizes and saves the predictions made by the model, alongside the components of the original dataset for comparison.

**GAN Training and Prediction Workflow**:

The model, specified by KL_CPD, is trained on a time series dataset. It first optionally reduces the dimensionality of the data using SVD.The training involves alternating between updating the discriminator (NetD) and generator (NetG) networks. The discriminator learns to differentiate between real and generated (synthetic) sequences, while the generator learns to produce sequences that are indistinguishable from real data.

The adversarial training process is guided by a loss function that incorporates MMD, a measure of the distance between the distributions of real and generated data.z

**Saving and Loading Models**: The framework supports saving the state of the model during training, which allows for training interruption and resumption without starting over.

**Utility and Visualization**: The code includes utilities for data preprocessing (e.g., HankelDataset for organizing time series data into training batches) and for visualizing training losses and prediction results.

Overall, this script is a comprehensive framework for training a GAN-based model on time series data for the purpose of detecting changes or anomalies. It leverages advanced techniques like RNNs for sequence data processing and MMD for distribution comparison, and it includes thoughtful considerations for usability, such as model saving/loading and visualization of results.

In [10]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio
import math
import torch
from torch.utils.data import Dataset


class HankelDataset(Dataset):
    def __init__(self, ts: np.array, p_wnd_dim:int=25, f_wnd_dim:int=10, sub_dim:int=1):
        """
        @param ts - timeseries
        @param p_wnd_dim - past window size
        @param f_wnd_dim - future window size
        @param sub_dim - Hankel matrix size
        """
        super().__init__()
        self.p_wnd_dim = p_wnd_dim
        self.f_wnd_dim = f_wnd_dim
        self.sub_dim = sub_dim

        if len(ts.shape) == 1: 
            ts = np.expand_dims(ts, -1)
        self.Y = ts # Timeseries
        self.T, self.D = ts.shape # T: time length; D: variable dimension
        self.var_dim = self.D * self.sub_dim

        self.Y_hankel = self.ts_to_hankel(self.Y)

    # prepare subspace data (Hankel matrix)
    def ts_to_hankel(self, ts):
        # T x D x sub_dim
        Y_hankel = np.zeros((self.T, self.D, self.sub_dim))
        for t in range(self.sub_dim, self.T):
            for d in range(self.D):
                Y_hankel[t, d, :] = ts[t-self.sub_dim+1:t+1, d].flatten()

        # Y_hankel is now T x (Dxsub_dim)
        Y_hankel = Y_hankel.reshape(self.T, -1)
        return Y_hankel


    # convert augmented data in Hankel matrix to origin time series
    # input: X_f, whose shape is batch_size x seq_len x (D*sub_dim)
    # output: Y_t, whose shape is batch_size x D
    def hankel_to_ts(self, X_f):
        batch_size = X_f.shape[0]
        Y_t = X_f[:, 0, :].contiguous().view(batch_size, self.D, self.sub_dim)
        return Y_t[:, :, -1]

    def __len__(self):
        return self.T

    def __getitem__(self, idx):
        if idx < 0: 
            idx += len(self)
        assert(0 <= idx < len(self))
        data = np.concatenate([
            np.tile(self.Y_hankel[:1,:], (self.p_wnd_dim, 1)), # left padding
            self.Y_hankel[:, :], # timeseries
            np.tile(self.Y_hankel[-1:,:], (self.f_wnd_dim, 1)), # right padding
            ])
        return {
            'X_p': torch.from_numpy(data[idx:idx+self.p_wnd_dim, :]),
            'X_f': torch.from_numpy(data[idx+self.p_wnd_dim:idx+self.p_wnd_dim+self.f_wnd_dim, :]),
            'Y': torch.from_numpy(self.Y[min(max(idx, 0), self.T-1)])
        }


In [11]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from sklearn.utils.extmath import svd_flip, randomized_svd
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
#from .data import HankelDataset
import time
import os


In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Running on {device.upper()}')


Running on CPU


In [13]:
import os

# Define the paths
base_dir = 'paths'
SAVE_MODEL_DIR_PATH =  os.path.join(base_dir, 'models')
PREDS_DIR_PATH =  os.path.join(base_dir, 'predictions')

# Example: Save a dummy file in the predictions directory to test
test_file_path = os.path.join(PREDS_DIR_PATH, 'test.txt')
with open(test_file_path, 'w') as file:
    file.write('This is a test file.')

print(f'Test file saved to: {test_file_path}')



Test file saved to: paths\predictions\test.txt


In [14]:
def median_heuristic(X, beta=0.5):
    max_n = min(30000, X.shape[0])
    D2 = euclidean_distances(X[:max_n], squared=True)
    med_sqdist = np.median(D2[np.triu_indices_from(D2, k=1)])
    beta_list = [beta**2, beta**1, 1, (1.0/beta)**1, (1.0/beta)**2]
    return [med_sqdist * b for b in beta_list]

In [15]:
class NetG(nn.Module):
    def __init__(self, var_dim, RNN_hid_dim, num_layers: int = 1):
        super().__init__()
        self.var_dim = var_dim
        self.RNN_hid_dim = RNN_hid_dim

        self.rnn_enc_layer = nn.GRU(
            self.var_dim, self.RNN_hid_dim, num_layers=num_layers, batch_first=True)
        self.rnn_dec_layer = nn.GRU(
            self.var_dim, self.RNN_hid_dim, num_layers=num_layers, batch_first=True)
        self.fc_layer = nn.Linear(self.RNN_hid_dim, self.var_dim)

    def forward(self, X_p, X_f, noise):
        X_p_enc, h_t = self.rnn_enc_layer(X_p)
        X_f_shft = self.shft_right_one(X_f)
        hidden = h_t + noise
        Y_f, _ = self.rnn_dec_layer(X_f_shft, hidden)
        output = self.fc_layer(Y_f)
        return output

    def shft_right_one(self, X):
        X_shft = X.clone()
        X_shft[:, 0, :].data.fill_(0)
        X_shft[:, 1:, :] = X[:, :-1, :]
        return X_shft


class NetD(nn.Module):
    def __init__(self, var_dim, RNN_hid_dim, num_layers: int = 1):
        super(NetD, self).__init__()

        self.var_dim = var_dim
        self.RNN_hid_dim = RNN_hid_dim

        self.rnn_enc_layer = nn.GRU(
            self.var_dim, self.RNN_hid_dim, num_layers=num_layers, batch_first=True)
        self.rnn_dec_layer = nn.GRU(
            self.RNN_hid_dim, self.var_dim, num_layers=num_layers, batch_first=True)

    def forward(self, X):
        X_enc, _ = self.rnn_enc_layer(X)
        X_dec, _ = self.rnn_dec_layer(X_enc)
        return X_enc, X_dec


In [16]:
class KL_CPD(nn.Module):
    def __init__(self, D: int, critic_iters: int = 5,
                 lambda_ae: float = 1e-5, lambda_real: float = 1e-3,
                 p_wnd_dim: int = 3, f_wnd_dim: int = 2, sub_dim: int = 1, RNN_hid_dim: int = 15):
        super().__init__()
        self.p_wnd_dim = p_wnd_dim
        self.f_wnd_dim = f_wnd_dim
        self.sub_dim = sub_dim
        self.D = D
        self.var_dim = D * sub_dim
        self.critic_iters = critic_iters
        self.lambda_ae, self.lambda_real = lambda_ae, lambda_real
        self.RNN_hid_dim = RNN_hid_dim
        self.netD = NetD(self.var_dim, RNN_hid_dim)
        self.netG = NetG(self.var_dim, RNN_hid_dim)
        self.loss_g_list = []
        self.loss_d_list = []

    @property
    def device(self):
        return next(self.parameters()).device

    def __mmd2_loss(self, X_p_enc, X_f_enc):
        sigma_var = self.sigma_var

        # some constants
        n_basis = 1024
        gumbel_lmd = 1e+6
        cnst = math.sqrt(1. / n_basis)
        n_mixtures = sigma_var.size(0)
        n_samples = n_basis * n_mixtures
        batch_size, seq_len, nz = X_p_enc.size()

        # gumbel trick to get masking matrix to uniformly sample sigma
        # input: (batch_size*n_samples, nz)
        # output: (batch_size, n_samples, nz)
        def sample_gmm(W, batch_size):
            U = torch.FloatTensor(batch_size*n_samples,
                                  n_mixtures).uniform_().to(self.device)
            sigma_samples = F.softmax(U * gumbel_lmd, dim=1).matmul(sigma_var)
            W_gmm = W.mul(1. / sigma_samples.unsqueeze(1))
            W_gmm = W_gmm.view(batch_size, n_samples, nz)
            return W_gmm

        W = Variable(torch.FloatTensor(batch_size*n_samples,
                     nz).normal_(0, 1).to(self.device))
        # batch_size x n_samples x nz
        W_gmm = sample_gmm(W, batch_size)
        # batch_size x nz x n_samples
        W_gmm = torch.transpose(W_gmm, 1, 2).contiguous()
        # batch_size x seq_len x n_samples
        XW_p = torch.bmm(X_p_enc, W_gmm)
        # batch_size x seq_len x n_samples
        XW_f = torch.bmm(X_f_enc, W_gmm)
        z_XW_p = cnst * torch.cat((torch.cos(XW_p), torch.sin(XW_p)), 2)
        z_XW_f = cnst * torch.cat((torch.cos(XW_f), torch.sin(XW_f)), 2)
        batch_mmd2_rff = torch.sum((z_XW_p.mean(1) - z_XW_f.mean(1))**2, 1)
        return batch_mmd2_rff

    def forward(self, X_p: torch.Tensor, X_f: torch.Tensor):
        batch_size = X_p.size(0)

        X_p_enc, _ = self.netD(X_p)
        X_f_enc, _ = self.netD(X_f)
        Y_pred_batch = self.__mmd2_loss(X_p_enc, X_f_enc)

        return Y_pred_batch

    def predict(self, ts):
        dataset = HankelDataset(
            ts, self.p_wnd_dim, self.f_wnd_dim, self.sub_dim)
        dataloader = DataLoader(dataset, batch_size=8, shuffle=False)
        preds = []
        with torch.no_grad():
            for batch in dataloader:
                X_p, X_f = [batch[key].float().to(self.device)
                            for key in ['X_p', 'X_f']]
                pred_val = self.forward(X_p, X_f).cpu().detach().numpy()
                preds.append(pred_val)
        return np.concatenate(preds)

    def fit(self, ts, start_epoch, svd_method, components, epoches: int = 100, lr: float = 1e-2, weight_clip: float = .1, weight_decay: float = 0., momentum: float = 0., dataset_name=None):
        print('***** Training *****')
        # must be defined in fit() method
        optG_adam = torch.optim.AdamW(
            self.netG.parameters(), lr=lr, weight_decay=weight_decay)
        # lr_scheduler_g = lr_scheduler.CosineAnnealingLR(optG, T_max=epoches, eta_min=3e-5)
        optD_adam = torch.optim.AdamW(
            self.netD.parameters(), lr=lr, weight_decay=weight_decay)
        # lr_scheduler_d = lr_scheduler.CosineAnnealingLR(optD, T_max=epoches, eta_min=3e-5)
        optD_rmsprop = torch.optim.RMSprop(
            self.netD.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
        optG_rmsprop = torch.optim.RMSprop(
            self.netG.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
        

        dataset = HankelDataset(
            ts, self.p_wnd_dim, self.f_wnd_dim, self.sub_dim)
        dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
        sigma_list = median_heuristic(dataset.Y_hankel, beta=.5)
        self.sigma_var = torch.FloatTensor(sigma_list).to(self.device)

        # tbar = trange(epoches)
        for epoch in tqdm(range(start_epoch, epoches)):
            for batch in dataloader:
                # Fit critic
                for p in self.netD.parameters():
                    p.requires_grad = True
                for p in self.netD.rnn_enc_layer.parameters():
                    p.data.clamp_(-weight_clip, weight_clip)
                # (D_mmd2_mean, mmd2_real_mean, real_L2_loss, fake_L2_loss) = self._optimizeD(batch, optD)
                # train after every 15 epochs
                # if epoch % 15 == 0:
                self._optimizeD(batch, optD_rmsprop)
                # G_mmd2_mean = 0
                # if np.random.choice(np.arange(self.critic_iters)) == 0:
                # Fit generator
                for p in self.netD.parameters():
                    p.requires_grad = False  # to avoid computation
                # G_mmd2_mean = self._optimizeG(batch, optG)
                self._optimizeG(batch, optG_rmsprop)
            
            optD_adam.step()
            optG_adam.step()          

            # saving model dict to file after every 5 epochs
            if dataset_name:
                if epoch % 2 == 0:
                    torch.save(self.netD.state_dict(
                    ), f'{SAVE_MODEL_DIR_PATH}/{dataset_name}/{svd_method}_{components}/netd_{epoch}.pt')
                    torch.save(self.netG.state_dict(
                    ), f'{SAVE_MODEL_DIR_PATH}/{dataset_name}/{svd_method}_{components}/netg_{epoch}.pt')
        print('***** Plotting losses for Generator and Discriminator models *****')
        self.plot_losses(reduction_method=svd_method,
                         components=components, dataset_name=dataset_name)
        # print('[%5d/%5d] D_mmd2 %.4e G_mmd2 %.4e mmd2_real %.4e real_L2 %.6f fake_L2 %.6f'
        #   % (epoch+1, epoches, D_mmd2_mean, G_mmd2_mean, mmd2_real_mean, real_L2_loss, fake_L2_loss))

    def _optimizeG(self, batch, opt, lr_scheduler=None, grad_clip: int = 5):
        X_p, X_f = [batch[key].float().to(self.device)
                    for key in ['X_p', 'X_f']]
        batch_size = X_p.size(0)

        # real data
        X_f_enc, X_f_dec = self.netD(X_f)

        # fake data
        noise = torch.FloatTensor(1, batch_size, self.RNN_hid_dim).uniform_(-1, 1).to(self.device)
        # noise = torch.FloatTensor(1, batch_size, self.RNN_hid_dim).normal_(0, 1).to(self.device)
        noise = Variable(noise)
        Y_f = self.netG(X_p, X_f, noise)
        Y_f_enc, Y_f_dec = self.netD(Y_f)

        # batchwise MMD2 loss between X_f and Y_f
        G_mmd2 = self.__mmd2_loss(X_f_enc, Y_f_enc)

        # update netG
        self.netG.zero_grad()
        lossG = G_mmd2.mean()
        # lossG = 0.0 * G_mmd2.mean()
        self.loss_g_list.append(lossG.data.item())
        lossG.backward()

        torch.nn.utils.clip_grad_norm_(self.netG.parameters(), grad_clip)

        opt.step()
        if lr_scheduler:
            lr_scheduler.step()

        # return G_mmd2.mean().data.item()

    def _optimizeD(self, batch, opt, lr_scheduler=None, grad_clip: int = 5):
        X_p, X_f, Y_true = [batch[key].float().to(self.device)
                            for key in ['X_p', 'X_f', 'Y']]
        batch_size = X_p.size(0)

        # real data
        X_p_enc, X_p_dec = self.netD(X_p)
        X_f_enc, X_f_dec = self.netD(X_f)

        # fake data
        noise = torch.FloatTensor(1, batch_size, self.netG.RNN_hid_dim).uniform_(-1, 1).to(self.device)
        # noise = torch.FloatTensor(1, batch_size, self.netG.RNN_hid_dim).normal_(0, 1).to(self.device)
        noise = Variable(noise)  # total freeze netG
        torch.no_grad()
        Y_f = Variable(self.netG(X_p, X_f, noise).data)
        Y_f_enc, Y_f_dec = self.netD(Y_f)

        # batchwise MMD2 loss between X_f and Y_f
        D_mmd2 = self.__mmd2_loss(X_f_enc, Y_f_enc)

        # batchwise MMD loss between X_p and X_f
        mmd2_real = self.__mmd2_loss(X_p_enc, X_f_enc)

        # reconstruction loss
        real_L2_loss = torch.mean((X_f - X_f_dec)**2)
        fake_L2_loss = torch.mean((Y_f - Y_f_dec)**2)

        # update netD
        self.netD.zero_grad()
        lossD = D_mmd2.mean() - self.lambda_ae * (real_L2_loss + fake_L2_loss) - \
            self.lambda_real * mmd2_real.mean()
        lossD = -lossD
        self.loss_d_list.append(lossD.data.item())
        lossD.backward()

        torch.nn.utils.clip_grad_norm_(self.netD.parameters(), grad_clip)

        opt.step()
        if lr_scheduler:
            lr_scheduler.step()

        # return D_mmd2.mean().data.item(), mmd2_real.mean().data.item(), real_L2_loss.data.item(), fake_L2_loss.data.item()

    def plot_losses(self, reduction_method: str, components: int, dataset_name: str):
        curr_time = time.strftime("%Y_%m_%d_%H_%M_%S")

        plt.figure(figsize=(10, 5))
        plt.plot(self.loss_g_list, label='loss_g')
        plt.plot(self.loss_d_list, label='loss_d')
        plt.legend()
        plt.savefig(
            f'{PREDS_DIR_PATH}/{curr_time}_{reduction_method}_{components}_{dataset_name}_loss.png')
        plt.show()

In [17]:
def svd_wrapper(Y, k, method='svds'):
    if method == 'svds':
        Ut, St, Vt = svds(Y, k)
        idx = np.argsort(St)[::-1]
        St = St[idx]  # have issue with sorting zero singular values
        Ut, Vt = svd_flip(Ut[:, idx], Vt[idx])
    elif method == 'random':
        Ut, St, Vt = randomized_svd(Y, k, random_state=0)
    else:
        Ut, St, Vt = np.linalg.svd(Y, full_matrices=False)
        # now truncate it to k
        Ut = Ut[:, :k]
        St = np.diag(St[:k])
        Vt = Vt[:k, :]

    return Ut, St, Vt


In [18]:
def get_reduced_data(dataset, components, svd_method):
    print(f'***** Original dataset shape: {dataset.shape} *****')
    X, _, _ = svd_wrapper(dataset, components, method=svd_method)
    print(f'***** Reduced dataset shape: {X.shape} *****')
    return X



In [19]:
def train_and_pred_dataset(dataset, dataset_name, svd_method, components, preload_model=False):
    '''If dataset name is not None, then save the model dict to file after each epoch. If preload_model is True, then load the model from file and continue training'''
    dimension = dataset.shape[1]
    start_epoch = 0
    model = KL_CPD(dimension).to(device)

    # get model state dict from the file
    if dataset_name:
        # check if folder exists if not then create it
        model_folder_path = f'{SAVE_MODEL_DIR_PATH}/{dataset_name}/{svd_method}_{components}/'
        if not os.path.exists(model_folder_path):
            os.makedirs(model_folder_path)
        dir_files = os.listdir(model_folder_path)
        print('-----------------', len(dir_files))
        # check if folder is empty and check if reset_model_folder is False
        if len(dir_files) > 0 and preload_model:
            # sort files that starts with netd and netg
            netd_files = [
                file for file in dir_files if file.startswith('netd')]
            netg_files = [
                file for file in dir_files if file.startswith('netg')]

            # sort files with latest timestamp
            netd_files = sorted(netd_files, key=lambda x: os.path.getmtime(
                model_folder_path + x), reverse=True)
            netg_files = sorted(netg_files, key=lambda x: os.path.getmtime(
                model_folder_path + x), reverse=True)

            # load the latest file from netd and netg
            net_d_params = torch.load(model_folder_path + netd_files[0])
            net_g_params = torch.load(model_folder_path + netg_files[0])

            # load params to model
            model.netD.load_state_dict(net_d_params)
            model.netG.load_state_dict(net_g_params)

            # get the epoch number from the model file name. if epoch number is different then choose the min epoch number
            start_epoch = min(int(netd_files[0].split(
                '_')[-1].split('.')[0]), int(netg_files[0].split('_')[-1].split('.')[0]))
            print(
                f'***** Loaded model from file: {svd_method}_{components}/{netd_files[0]} and {svd_method}_{components}/{netg_files[0]} with epoch {start_epoch} *****')

    model.fit(dataset, dataset_name=dataset_name, start_epoch=start_epoch,
              svd_method=svd_method, components=components)
    predictions = model.predict(dataset)
    return predictions



In [20]:
def save_preds(dataset, predictions, reduction_method, dataset_name, skip_components=0, save_preds=True):
    print('***** Saving Predictions *****')
    components = dataset.shape[1]
    # get the min and max values for y-axis
    min_y = float('inf')
    max_y = float('-inf')
    for i in range(components):
        if skip_components == i+1:
            continue
        min_y = min(min_y, min(dataset[:, i]))
        max_y = max(max_y, max(dataset[:, i]))

    for i in range(components):
        if skip_components == i+1:
            continue
        plt.subplot(components+1, 1, i+1)
        plt.plot(dataset[:, i])
        plt.title(f'Component {i+1}')
        plt.ylim([min_y-0.2, max_y+0.2])
        plt.subplot(components+1, 1, components+1)

    plt.plot(predictions)
    plt.title('MMD')
    plt.suptitle(
        f'{reduction_method} with {components-skip_components} component(s) visualization')
    plt.tight_layout()
    if save_preds:
        curr_time = time.strftime("%Y_%m_%d_%H_%M_%S")
        plt.savefig(
            f'{PREDS_DIR_PATH}/{curr_time}_{reduction_method}_{components-skip_components}_{dataset_name}.png')
    plt.show()
    print('***** DONE *****')

In [21]:
from enum import Enum
class DatasetName(Enum):
    PROTEIN_1BYZ = '1byz_md.xyz', [100, 56, 3]
    
def check_valid_dataset(dataset_name):
    assert isinstance(dataset_name, str) and dataset_name in DatasetName.__members__, 'Dataset name must be a string'
    return True

def get_details(dataset_name):
    check_valid_dataset(dataset_name)
    data = DatasetName[dataset_name].value
    return data


def check_pkl_file(file_name):
    file_arr = file_name.split('.')
    pkl_file_name = '_'.join(file_arr) + '.pkl'
    files = os.listdir(DATASET_DIR_PATH)

    return pkl_file_name in files, f'{DATASET_DIR_PATH}/{pkl_file_name}'


In [22]:
def get_coordinates(dataset_name):
    file_name, shape = get_details(dataset_name)
    is_present, pickle_file_name = check_pkl_file(file_name)

    if is_present:
        coordinates = torch.load(pickle_file_name)
        return coordinates
    
    xyz = open(f'{DATASET_DIR_PATH}/{file_name}')
    
    coordinates = []
    for row in xyz:
        x, y, z = row.split(',')
        coordinates.append([float(x), float(y), float(z)])

    coordinates = np.array(coordinates)
    coordinates = coordinates.reshape(shape)

    (x1, y1, z1) = coordinates.shape
    coordinates = coordinates.reshape(x1, y1*z1, order='C')

    torch.save(coordinates, pickle_file_name)

    return coordinates

In [23]:
# def train(dataset_name, svd_method, components):
#     data = get_coordinates(dataset_name)
#     data_reduced = get_reduced_data(data, components, svd_method)
#     preds = train_and_pred_dataset(data_reduced, dataset_name.lower(), svd_method, components)
#     save_preds(data_reduced, preds, svd_method, dataset_name.lower())

# if __name__ == "__main__":
#     dataset_name = '1byz_md.xyz'
#     svd_method = 'random'
#     components = 2
#     train(dataset_name, svd_method, components)

In [24]:
from pathlib import Path

# Assuming your .xyz file is located directly in the "1byz_md.xyz" folder,
# and your directory structure is "/pscratch/sd/s/saik1999/KLCPD/protein_data/1byz_md.xyz/"
DATASET_DIR_PATH = 'paths/data' # Path('paths/data')

# The file name should be just the name of the file, not the entire path
file_name = '1byz_md.xyz'


In [25]:
DATASET_DIR_PATH

'paths/data'

In [26]:
# DATASET_DIR_PATH='pscratch/sd/s/saik1999/KLCPD/protein_data/1byz_md.xyz'

In [27]:
f"{DATASET_DIR_PATH}/{file_name}"

'paths/data/1byz_md.xyz'

In [35]:
import os
import numpy as np
import torch
from enum import Enum
from pathlib import Path

# Assuming environment variables are set for the dataset directory path
# DATASET_DIR_PATH = Path(os.getenv('DATASET_DIR_PATH', '.'))

class DatasetName(Enum):
    PROTEIN_1BYZ = '1byz_md.xyz', (100, 56, 3)

def get_coordinates(dataset_name):
    file_name, shape = DatasetName[dataset_name].value # dataset_enum.value
    pkl_file_name = f"{DATASET_DIR_PATH}/{file_name.split('.')[0]}.pkl"

    # if pkl_file_name.exists():
    #     print("Loading coordinates from pickle file.")
    #     return torch.load(str(1pkl_file_name))
    
    print("Reading XYZ file and creating pickle file.")
    coordinates = []
    with open(f"{DATASET_DIR_PATH}/{file_name}", 'r') as xyz_file:
        # Skip the first line that possibly contains number of atoms
        # next(xyz_file)
        # Skip the second line that might be a comment
        # next(xyz_file)
        for row in xyz_file:
            parts = row.split(',')
            # print(parts)
            if len(parts) == 4:  # Expecting: Atom X Y Z
                # We are only interested in coordinates here, ignoring the atom type
                coordinates.append(list(map(float, parts[1:])))
    
    # Assuming the 'shape' tuple contains the expected dimensions in your data
    # For example, shape could be (1000, 3) for 1000 points in 3D space
    L = len(coordinates)
    D1 = 100
    D2 = 56 # L // D1
    coordinates = coordinates[:D1*D2]
    coordinates = np.array(coordinates).reshape([D1, D2, 3])
    coordinates = coordinates.reshape([D1 * D2, 3])
    torch.save(coordinates, str(pkl_file_name))

    return coordinates
def main():
    # Choose dataset
    dataset_name = 'PROTEIN_1BYZ'
    # print(dataset_enum, '--------')

    # Get coordinates
    coordinates = get_coordinates(dataset_name)

    # Here you would add your model training code
    print(f"Loaded data shape: {coordinates.shape}")

if __name__ == "__main__":
    main()


Reading XYZ file and creating pickle file.
Loaded data shape: (5600, 3)


In [36]:
def train(dataset_name, svd_method, components):

    preload_model_name = f"PRE_LOAD_{dataset_name.upper()}_MODEL"
    # convert to boolean as env variable are always string
    PRE_LOAD_PROTEIN_1FME_MODEL = os.getenv(preload_model_name, False) == 'True'
    data = get_coordinates(dataset_name)
    data_reduced = get_reduced_data(data, components, svd_method)
    preds = train_and_pred_dataset(data_reduced, dataset_name.lower(), svd_method, components, preload_model=PRE_LOAD_PROTEIN_1FME_MODEL)
    save_preds(data_reduced, preds, svd_method, dataset_name.lower())


In [ ]:
dataset_name = 'PROTEIN_1BYZ'
svd_method = 'random'
components = 2
train(dataset_name, svd_method, components)

Reading XYZ file and creating pickle file.
***** Original dataset shape: (5600, 3) *****
***** Reduced dataset shape: (5600, 2) *****
----------------- 0
***** Training *****


 85%|███████████████████████████████████████████████████████████████████▏           | 85/100 [2:15:04<23:28, 93.91s/it]

In [ ]:
from functools import partial
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope


In [ ]:
class KL_CPD(nn.Module):
    def __init__(self, D: int, critic_iters: int = 5,
                 lambda_ae: float = 1e-5, lambda_real: float = 1e-3,
                 p_wnd_dim: int = 3, f_wnd_dim: int = 2, sub_dim: int = 1, RNN_hid_dim: int = 15):
        super().__init__()
        self.p_wnd_dim = p_wnd_dim
        self.f_wnd_dim = f_wnd_dim
        self.sub_dim = sub_dim
        self.D = D
        self.var_dim = D * sub_dim
        self.critic_iters = critic_iters
        self.lambda_ae, self.lambda_real = lambda_ae, lambda_real
        self.RNN_hid_dim = RNN_hid_dim
        self.netD = NetD(self.var_dim, RNN_hid_dim)
        self.netG = NetG(self.var_dim, RNN_hid_dim)
        self.loss_g_list = []
        self.loss_d_list = []

    @property
    def device(self):
        return next(self.parameters()).device

    def __mmd2_loss(self, X_p_enc, X_f_enc):
        sigma_var = self.sigma_var

        # some constants
        n_basis = 1024
        gumbel_lmd = 1e+6
        cnst = math.sqrt(1. / n_basis)
        n_mixtures = sigma_var.size(0)
        n_samples = n_basis * n_mixtures
        batch_size, seq_len, nz = X_p_enc.size()

        # gumbel trick to get masking matrix to uniformly sample sigma
        # input: (batch_size*n_samples, nz)
        # output: (batch_size, n_samples, nz)
        def sample_gmm(W, batch_size):
            U = torch.FloatTensor(batch_size*n_samples,
                                  n_mixtures).uniform_().to(self.device)
            sigma_samples = F.softmax(U * gumbel_lmd, dim=1).matmul(sigma_var)
            W_gmm = W.mul(1. / sigma_samples.unsqueeze(1))
            W_gmm = W_gmm.view(batch_size, n_samples, nz)
            return W_gmm

        W = Variable(torch.FloatTensor(batch_size*n_samples,
                     nz).normal_(0, 1).to(self.device))
        # batch_size x n_samples x nz
        W_gmm = sample_gmm(W, batch_size)
        # batch_size x nz x n_samples
        W_gmm = torch.transpose(W_gmm, 1, 2).contiguous()
        # batch_size x seq_len x n_samples
        XW_p = torch.bmm(X_p_enc, W_gmm)
        # batch_size x seq_len x n_samples
        XW_f = torch.bmm(X_f_enc, W_gmm)
        z_XW_p = cnst * torch.cat((torch.cos(XW_p), torch.sin(XW_p)), 2)
        z_XW_f = cnst * torch.cat((torch.cos(XW_f), torch.sin(XW_f)), 2)
        batch_mmd2_rff = torch.sum((z_XW_p.mean(1) - z_XW_f.mean(1))**2, 1)
        return batch_mmd2_rff

    def forward(self, X_p: torch.Tensor, X_f: torch.Tensor):
        batch_size = X_p.size(0)

        X_p_enc, _ = self.netD(X_p)
        X_f_enc, _ = self.netD(X_f)
        Y_pred_batch = self.__mmd2_loss(X_p_enc, X_f_enc)

        return Y_pred_batch


    def predict(self, ts):
        dataset = HankelDataset(
            ts, self.p_wnd_dim, self.f_wnd_dim, self.sub_dim)
        dataloader = DataLoader(dataset, batch_size=8, shuffle=False)
        preds = []
        with torch.no_grad():
            for batch in dataloader:
                X_p, X_f = [batch[key].float().to(self.device)
                            for key in ['X_p', 'X_f']]
                pred_val = self.forward(X_p, X_f).cpu().detach().numpy()
                preds.append(pred_val)
        return np.concatenate(preds)

    def fit(self, ts, start_epoch, svd_method, components, epoches: int = 100, lr: float = 1e-2, weight_clip: float = .1, weight_decay: float = 0., momentum: float = 0., dataset_name=None):
        # print('***** Training *****')
        # must be defined in fit() method
        optG_adam = torch.optim.AdamW(
            self.netG.parameters(), lr=lr, weight_decay=weight_decay)
        # lr_scheduler_g = lr_scheduler.CosineAnnealingLR(optG, T_max=epoches, eta_min=3e-5)
        optD_adam = torch.optim.AdamW(
            self.netD.parameters(), lr=lr, weight_decay=weight_decay)
        # lr_scheduler_d = lr_scheduler.CosineAnnealingLR(optD, T_max=epoches, eta_min=3e-5)
        optD_rmsprop = torch.optim.RMSprop(
            self.netD.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
        optG_rmsprop = torch.optim.RMSprop(
            self.netG.parameters(), lr=lr, weight_decay=weight_decay, momentum=momentum)
        

        dataset = HankelDataset(
            ts, self.p_wnd_dim, self.f_wnd_dim, self.sub_dim)
        dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
        sigma_list = median_heuristic(dataset.Y_hankel, beta=.5)
        self.sigma_var = torch.FloatTensor(sigma_list).to(self.device)

        # tbar = trange(epoches)

        for epoch in range(start_epoch, epoches):
            for batch in dataloader:
                # Fit critic
                for p in self.netD.parameters():
                    p.requires_grad = True
                for p in self.netD.rnn_enc_layer.parameters():
                    p.data.clamp_(-weight_clip, weight_clip)
                # (D_mmd2_mean, mmd2_real_mean, real_L2_loss, fake_L2_loss) = self._optimizeD(batch, optD)
                # train after every 15 epochs
                # if epoch % 15 == 0:
                self._optimizeD(batch, optD_rmsprop)
                # G_mmd2_mean = 0
                # if np.random.choice(np.arange(self.critic_iters)) == 0:
                # Fit generator
                for p in self.netD.parameters():
                    p.requires_grad = False  # to avoid computation
                # G_mmd2_mean = self._optimizeG(batch, optG)
                self._optimizeG(batch, optG_rmsprop)
            
            optD_adam.step()
            optG_adam.step()          

    def minimize(self, ts):
        dataset = HankelDataset(
            ts, self.p_wnd_dim, self.f_wnd_dim, self.sub_dim)
        dataloader = DataLoader(dataset, batch_size=8, shuffle=False)
        optG_losses = []
        optD_losses = []
        with torch.no_grad():
            for batch in dataloader:
                X_p, X_f = [batch[key].float().to(self.device)
                            for key in ['X_p', 'X_f']]
                 batch_size = X_p.size(0)

                # real data
                X_f_enc, X_f_dec = self.netD(X_f)
        
                # fake data
                noise = torch.FloatTensor(1, batch_size, self.RNN_hid_dim).uniform_(-1, 1).to(self.device)
                # noise = torch.FloatTensor(1, batch_size, self.RNN_hid_dim).normal_(0, 1).to(self.device)
                noise = Variable(noise)
                Y_f = self.netG(X_p, X_f, noise)
                Y_f_enc, Y_f_dec = self.netD(Y_f)
                G_mmd2 = self.__mmd2_loss(X_f_enc, Y_f_enc)
                optG_losses.append(G_mmd2.mean().data.item())

                X_p_enc, X_p_dec = self.netD(X_p)
                X_f_enc, X_f_dec = self.netD(X_f)
        
                # fake data
                noise = torch.FloatTensor(1, batch_size, self.netG.RNN_hid_dim).uniform_(-1, 1).to(self.device)
                # noise = torch.FloatTensor(1, batch_size, self.netG.RNN_hid_dim).normal_(0, 1).to(self.device)
                noise = Variable(noise)  # total freeze netG
                torch.no_grad()
                Y_f = Variable(self.netG(X_p, X_f, noise).data)
                Y_f_enc, Y_f_dec = self.netD(Y_f)
        
                # batchwise MMD2 loss between X_f and Y_f
                D_mmd2 = self.__mmd2_loss(X_f_enc, Y_f_enc)
        
                # batchwise MMD loss between X_p and X_f
                mmd2_real = self.__mmd2_loss(X_p_enc, X_f_enc)
                optD_losses(mmd2_real.mean().item().data())
        return np.mean(optG_losses) + np.mean(optD_losses)

    def _optimizeG(self, batch, opt, lr_scheduler=None, grad_clip: int = 5):
        X_p, X_f = [batch[key].float().to(self.device)
                    for key in ['X_p', 'X_f']]
        batch_size = X_p.size(0)

        # real data
        X_f_enc, X_f_dec = self.netD(X_f)

        # fake data
        noise = torch.FloatTensor(1, batch_size, self.RNN_hid_dim).uniform_(-1, 1).to(self.device)
        # noise = torch.FloatTensor(1, batch_size, self.RNN_hid_dim).normal_(0, 1).to(self.device)
        noise = Variable(noise)
        Y_f = self.netG(X_p, X_f, noise)
        Y_f_enc, Y_f_dec = self.netD(Y_f)

        # batchwise MMD2 loss between X_f and Y_f
        G_mmd2 = self.__mmd2_loss(X_f_enc, Y_f_enc)

        # update netG
        self.netG.zero_grad()
        lossG = G_mmd2.mean()
        # lossG = 0.0 * G_mmd2.mean()
        self.loss_g_list.append(lossG.data.item())
        lossG.backward()

        torch.nn.utils.clip_grad_norm_(self.netG.parameters(), grad_clip)

        opt.step()
        if lr_scheduler:
            lr_scheduler.step()

        # return G_mmd2.mean().data.item()

    def _optimizeD(self, batch, opt, lr_scheduler=None, grad_clip: int = 5):
        X_p, X_f, Y_true = [batch[key].float().to(self.device)
                            for key in ['X_p', 'X_f', 'Y']]
        batch_size = X_p.size(0)

        # real data
        X_p_enc, X_p_dec = self.netD(X_p)
        X_f_enc, X_f_dec = self.netD(X_f)

        # fake data
        noise = torch.FloatTensor(1, batch_size, self.netG.RNN_hid_dim).uniform_(-1, 1).to(self.device)
        # noise = torch.FloatTensor(1, batch_size, self.netG.RNN_hid_dim).normal_(0, 1).to(self.device)
        noise = Variable(noise)  # total freeze netG
        torch.no_grad()
        Y_f = Variable(self.netG(X_p, X_f, noise).data)
        Y_f_enc, Y_f_dec = self.netD(Y_f)

        # batchwise MMD2 loss between X_f and Y_f
        D_mmd2 = self.__mmd2_loss(X_f_enc, Y_f_enc)

        # batchwise MMD loss between X_p and X_f
        mmd2_real = self.__mmd2_loss(X_p_enc, X_f_enc)

        # reconstruction loss
        real_L2_loss = torch.mean((X_f - X_f_dec)**2)
        fake_L2_loss = torch.mean((Y_f - Y_f_dec)**2)

        # update netD
        self.netD.zero_grad()
        lossD = D_mmd2.mean() - self.lambda_ae * (real_L2_loss + fake_L2_loss) - \
            self.lambda_real * mmd2_real.mean()
        lossD = -lossD
        self.loss_d_list.append(lossD.data.item())
        lossD.backward()

        torch.nn.utils.clip_grad_norm_(self.netD.parameters(), grad_clip)

        opt.step()
        if lr_scheduler:
            lr_scheduler.step()

#        return D_mmd2.mean().data.item() + mmd2_real.mean().data.item() +  real_L2_loss.data.item() + fake_L2_loss.data.item()


def svd_wrapper(Y, k, method='svds'):
    if method == 'svds':
        Ut, St, Vt = svds(Y, k)
        idx = np.argsort(St)[::-1]
        St = St[idx]  # have issue with sorting zero singular values
        Ut, Vt = svd_flip(Ut[:, idx], Vt[idx])
    elif method == 'random':
        Ut, St, Vt = randomized_svd(Y, k, random_state=0)
    else:
        Ut, St, Vt = np.linalg.svd(Y, full_matrices=False)
        # now truncate it to k
        Ut = Ut[:, :k]
        St = np.diag(St[:k])
        Vt = Vt[:k, :]

    return Ut, St, Vt


def get_reduced_data(dataset, components, svd_method):
    print(f'***** Original dataset shape: {dataset.shape} *****')
    X, _, _ = svd_wrapper(dataset, components, method=svd_method)
    print(f'***** Reduced dataset shape: {X.shape} *****')
    return X


def train_and_pred_dataset(dataset, dataset_name, svd_method, components, preload_model=False):
    '''If dataset name is not None, then save the model dict to file after each epoch. If preload_model is True, then load the model from file and continue training'''
    dimension = dataset.shape[1]
    start_epoch = 0
    model = KL_CPD(dimension).to(device)


    model.fit(dataset, dataset_name=dataset_name, start_epoch=start_epoch,
              svd_method=svd_method, components=components)
    predictions = model.predict(dataset)
    return predictions


In [ ]:
# The parameter space range
params = {
    "lambda_ae": hp.uniform("lambda_ae", 0.000001, 0.1),
    "lambda_real": hp.uniform("lambda_real", 0.000001, 0.1),
    "p_wnd_dim": scope.int(hp.quniform("p_wnd_dim", 1, 5, 1)),
    "f_wnd_dim": scope.int(hp.quniform("f_wnd_dim", 1, 5, 1)),
    "sub_dim": scope.int(hp.quniform("sub_dim", 1, 5, 1)),
    "RNN_hid_dim": scope.int(hp.quniform("RNN_hid_dim", 1, 15, 1)),
    "svd_method": hp.choice("svd_method", ["svds", "random", "regular", None]),
    "component": hp.choice("component", [1, 2]),
}

In [ ]:
preload_model_name = f"PRE_LOAD_{dataset_name.upper()}_MODEL"
# convert to boolean as env variable are always string
PRE_LOAD_PROTEIN_1FME_MODEL = os.getenv(preload_model_name, False) == 'True'

In [ ]:
# The optimization function to be minimized
def optimize(params):

    data = get_coordinates(dataset_name)
    if params['svd_method']:
        data = get_reduced_data(data, params['components'], params['svd_method'])
    
    dimension = data.shape[1]
    model = KL_CPD(
        dimension,
        lambda_ae = params['lambda_ae'],
        lambda_real = params['lambda_real'],
        p_wnd_dim = params['p_wnd_dim'],
        f_wnd_dim = params['f_wnd_dim'],
        sub_dim = params['sub_dim'],
        RNN_hid_dim = params['RNN_hid_dim']
    ).to(device)

    return  model.minimize(data)

    

In [ ]:
optimization_function = partial(
    optimize
 )
 # initialize trials to keep logging information
 trials = Trials()

 # run hyperopt with 10 trials
 hopt = fmin(
     fn=optimization_function,
     space=param_space,
     algo=tpe.suggest,
     max_evals=10,
     trials=trials
 )